**Общая информация**

**Срок сдачи:** 21 октября 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter 

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree
from scipy.stats import mode

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [2]:
def distCosine (vecA, vecB):
    def dotProduct (vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB:
                d += vecA[dim]*vecB[dim]
        return d
    return np.dot(vecA,vecB) / np.sqrt(dotProduct(vecA,vecA)) / np.sqrt(dotProduct(vecB,vecB))


class MyKNeighborsClassifier(BaseEstimator): 
    x_train = np.array([])
    y_train = np.array([])
    n_neighbors = 0
    algorithm = "brute"
    distance = "evklid"
    def __init__(self, n_neighbors, algorithm='brute', distance = "evklid"):
        self.n_neighbors = n_neighbors
        if (algorithm == 'kd_tree'):
            self.algorithm = "kd_tree"
        if (distance == "cos"):
            self.distance = "cos"
            pass

    def fit(self, X, y):
        self.x_train = X
        self.y_train = y
        pass
    
    def predict(self, X):
        if (self.algorithm == "brute"):
            predictions = []
            for x_test in X:
                if (self.distance == "evklid"):
                    distances = np.sum(np.square(self.x_train - x_test), axis=1)
                else:
                    distances = pairwise_distances(self.x_train, np.repeat(x_test.reshape(1, -1), self.x_train.shape[0], axis=0), metric='cosine')
                min_index_proto = distances.argsort()
                min_index = min_index_proto[:self.n_neighbors]
                y_pred_k = y_train[min_index]
                y_pred = np.asarray(np.unique(y_pred_k, return_counts=True))
                y_pred = np.sort(y_pred, axis=1)
                predictions.append(y_pred[0][0])
        else:
            predictions = []
            tree = KDTree(self.x_train)
            dist, ind = tree.query(X, k=self.n_neighbors)
            y_pred_k = y_train[ind]

            val, count = mode(y_pred_k, axis=1)
            predictions = val.reshape(-1)
            
        return np.array(predictions)
        
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [4]:
iris = datasets.load_iris()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [6]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')


In [7]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [8]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [9]:
%time clf.fit(X_train, y_train)

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 658 µs


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [10]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


In [11]:
%time clf.predict(X_test)

CPU times: user 8.06 ms, sys: 14 µs, total: 8.07 ms
Wall time: 5.32 ms


array([1, 0, 1, 0, 1, 2, 2, 2, 0, 0, 1, 2, 0, 1, 1])

In [12]:
%time my_clf.predict(X_test)

CPU times: user 2.39 ms, sys: 469 µs, total: 2.86 ms
Wall time: 2.03 ms


array([1, 0, 1, 0, 1, 2, 2, 2, 0, 0, 1, 2, 0, 1, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [13]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [15]:
%time clf.fit(X_train, y_train)

CPU times: user 1.38 ms, sys: 1.09 ms, total: 2.47 ms
Wall time: 26.7 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [16]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs


In [17]:
%time clf.predict(X_test)

CPU times: user 3.66 ms, sys: 0 ns, total: 3.66 ms
Wall time: 2.37 ms


array([0, 1, 1, 2, 0, 2, 1, 2, 2, 0, 0, 2, 1, 0, 1])

In [18]:
%time my_clf.predict(X_test)

CPU times: user 2.25 ms, sys: 0 ns, total: 2.25 ms
Wall time: 1.72 ms


array([0, 1, 1, 2, 0, 2, 1, 2, 2, 0, 0, 2, 1, 0, 1])

In [19]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [4]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [5]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=5)
X = vectorizer.fit_transform(data).toarray()
Y = target
X = X[:,:100:]
print(X.shape)
print(Y.shape)


(11314, 100)
(11314,)


*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    X_tr_va, X_test = X[train_index], X[test_index]
    y_tr_va, y_test = Y[train_index], Y[test_index]
    X_train, X_val, y_train, y_val = train_test_split(X_tr_va, y_tr_va)


Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [11]:
from sklearn.preprocessing import StandardScaler

In [14]:
a = np.array([1, 2, 3, 4])
b = np.array([])
for k in  range(4):
    np.append(b, a[k])
print(b)

[]


In [12]:
max_accuracy = 0
k1 = 0
accuracy_vector = np.array([])
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_val, y_train, y_val = X[train_index], X[test_index], Y[train_index], Y[test_index]
    X_train = StandardScaler().fit(X_train).transform(X_train)
    X_val = StandardScaler().fit(X_val).transform(X_val)
    accuracy_vector = np.array([])
    for k in range(1, 25):
        my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute')
        my_clf.fit(X_train, y_train)
        my_clf_pred = my_clf.predict(X_val)
        accuracy = accuracy_score(y_val, my_clf_pred)
        np.append(accuracy_vector, accuracy)
        if (max_accuracy < accuracy):
            max_accuracy = accuracy
            k1 = k
print(k1, max_accuracy)
print(accuracy_vector)

1 0.0562185096791302
[]


In [13]:
from sklearn.metrics import pairwise_distances
max_accuracy = 0
k1 = 0
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_val, y_train, y_val = X[train_index], X[test_index], Y[train_index], Y[test_index]
    X_train = StandardScaler().fit(X_train).transform(X_train)
    X_val = StandardScaler().fit(X_val).transform(X_val)
    for k in range(1, 25):
        my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute', distance="cos")
        my_clf.fit(X_train, y_train)
        my_clf_pred = my_clf.predict(X_val)
        accuracy = accuracy_score(y_val, my_clf_pred)
        if (max_accuracy < accuracy):
            max_accuracy = accuracy
            k1 = k
print(k1, max_accuracy)

KeyboardInterrupt: 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=5)
X = vectorizer.fit_transform(data).toarray()
Y = target
X = X[:,:100:]
print(X.shape)
print(Y.shape)
max_accuracy = 0
k1 = 0
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_val, y_train, y_val = X[train_index], X[test_index], Y[train_index], Y[test_index]
    X_train = StandardScaler().fit(X_train).transform(X_train)
    X_val = StandardScaler().fit(X_val).transform(X_val)
    for k in range(1, 25):
        my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute')
        my_clf.fit(X_train, y_train)
        my_clf_pred = my_clf.predict(X_val)
        accuracy = accuracy_score(y_val, my_clf_pred)
        if (max_accuracy < accuracy):
            max_accuracy = accuracy
            k1 = k
print(k1, max_accuracy)

(11314, 100)
(11314,)
1 0.0562185096791302


Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [31]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [32]:
data = newsgroups['data']
target = newsgroups['target']
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=5)
X = vectorizer.fit_transform(data).toarray()
Y = target
X = X[:,:100:]
print(X.shape)
print(Y.shape)
max_accuracy = 0
k1 = 0
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_val, y_train, y_val = X[train_index], X[test_index], Y[train_index], Y[test_index]
    for k in range(1, 25):
        my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute')
        my_clf.fit(X_train, y_train)
        my_clf_pred = my_clf.predict(X_val)
        accuracy = accuracy_score(y_val, my_clf_pred)
        if (max_accuracy < accuracy):
            max_accuracy = accuracy
            k1 = k
print(k1, max_accuracy)


(7532, 100)
(7532,)
4 0.05856573705179283
